In [1]:
import requests
import json
import datetime
import time

In [2]:
BASE_URL = "http://localhost:8000"

Пробуем получить информацию о текущем пользователе без авторизации

In [3]:
response = requests.get(f"{BASE_URL}/users/me")
response.status_code, response.json()

(401, {'detail': 'Not authenticated'})

Регистрируем нового пользователя

In [4]:
payload = {"email": "user@example.com", "password": "string"}

response = requests.post(f"{BASE_URL}/register", data=json.dumps(payload))
response.status_code, response.json()

(200,
 {'id': 1,
  'email': 'user@example.com',
  'registered_at': '2025-03-24T17:37:40.965914'})

Пробуем зарегистрировать нового пользователя на уже существующий email

In [5]:
response = requests.post(f"{BASE_URL}/register", data=json.dumps(payload))
response.status_code, response.json()

(400, {'detail': 'This email is already taken'})

Авторизуемся (получаем токен)

In [6]:
response = requests.post(f"{BASE_URL}/login", data=json.dumps(payload))
token = response.json()["access_token"]
response.status_code

200

Еще раз пробуем получить информацию о текущем пользователе, но уже используем полученный токен

In [7]:
headers = {"Authorization": f"Bearer {token}"}

response = requests.get(f"{BASE_URL}/users/me", headers=headers)
response.status_code, response.json()

(200,
 {'id': 1,
  'email': 'user@example.com',
  'registered_at': '2025-03-24T17:37:40.965914'})

Создаем новую короткую ссылку

In [8]:
payload = {
  "original_url": "https://jsonplaceholder.typicode.com/posts/1"
}

response = requests.post(f"{BASE_URL}/links/shorten", headers=headers, data=json.dumps(payload))
response.status_code, response.json()

(200,
 {'original_url': 'https://jsonplaceholder.typicode.com/posts/1',
  'short_code': 'TM7GvKxa',
  'created_at': '2025-03-24T17:37:41.037365',
  'expires_at': '2025-04-23T17:37:41.037390'})

Тестируем переход по созданной ссылке

In [9]:
short_code = response.json()['short_code']

response = requests.get(f"{BASE_URL}/links/{short_code}")
response.status_code, response.json()

(200,
 {'userId': 1,
  'id': 1,
  'title': 'sunt aut facere repellat provident occaecati excepturi optio reprehenderit',
  'body': 'quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto'})

Создадим еще одну короткую ссылку с кастомным алиасом и укажем свою дату экспирации

In [10]:
custom_alias = "my_custom_alias"

payload = {
    "original_url": "https://github.com/",
    "custom_alias": custom_alias,
    "expires_at": datetime.datetime(2025, 5, 24).isoformat()
}

response = requests.post(f"{BASE_URL}/links/shorten", headers=headers, data=json.dumps(payload))
response.status_code, response.json()

(200,
 {'original_url': 'https://github.com/',
  'short_code': 'my_custom_alias',
  'created_at': '2025-03-24T17:37:41.037365',
  'expires_at': '2025-05-24T00:00:00'})

Проверим поиск по оригинальному URL

In [11]:
payload = {
    "original_url": "https://github.com"
}

response = requests.get(f"{BASE_URL}/links/search", headers=headers, params=payload)
response.status_code, response.json()

(200,
 [{'original_url': 'https://github.com/',
   'short_code': 'my_custom_alias',
   'created_at': '2025-03-24T17:37:41.037365',
   'expires_at': '2025-05-24T00:00:00'}])

Изменим оригинальный URL для короткой ссылки

In [12]:
params = {"new_url": "https://example.com/"}

response = requests.put(f"{BASE_URL}/links/{custom_alias}", params=params, headers=headers)
response.status_code, response.json()

(200,
 {'original_url': 'https://example.com/',
  'short_code': 'my_custom_alias',
  'created_at': '2025-03-24T17:37:41.037365',
  'expires_at': '2025-05-24T00:00:00'})

Удалим короткую ссылку

In [13]:
response = requests.delete(f"{BASE_URL}/links/{custom_alias}", headers=headers)
response.status_code, response.json()

(200, {'message': 'Short URL deleted'})

Для оставшейся (первой) короткой ссылки посмотрим статистику (уже должен быть 1 просмотр)

In [14]:
response = requests.get(f"{BASE_URL}/links/{short_code}/stats", headers=headers)
response.status_code, response.json()

(200,
 {'short_code': 'TM7GvKxa',
  'created_at': '2025-03-24T17:37:41.037365',
  'last_requested': '2025-03-24T17:37:41.041687',
  'num_requests': 1})

Проверим кэширование

In [15]:
# Проверяем, что на текущий момент кол-во запросов = 1
response = requests.get(f"{BASE_URL}/links/{short_code}/stats")
assert int(response.json()['num_requests']) == 1

# Делаем новый (2-й запрос)
# и сразу же после этого опять смотрим статистику,
# т.к. запрос закэширован, результат все еще остается = 1
requests.get(f"{BASE_URL}/links/{short_code}")

response = requests.get(f"{BASE_URL}/links/{short_code}/stats")
assert int(response.json()['num_requests']) == 1


# Ждем минуту (время жизни кэша)
# и проверяем, что эндпоинт выдаст актуальное значение = 2 
time.sleep(61)
response =  requests.get(f"{BASE_URL}/links/{short_code}/stats")
assert int(response.json()['num_requests']) == 2

Проверим, что изменение/удаление короткой ссылки доступно только ее создателю

In [16]:
# Зарегистрируем нового пользователя и залогинимся под ним
payload = {"email": "user1@example.com", "password": "string"}

requests.post(f"{BASE_URL}/register", data=json.dumps(payload))

token = requests.post(f"{BASE_URL}/login", data=json.dumps(payload)).json()["access_token"]
headers = {"Authorization": f"Bearer {token}"}

# Попробуем удалить ссылку, созданную первым пользователем
response = requests.delete(f"{BASE_URL}/links/{short_code}", headers=headers)
response.status_code, response.json()

(403, {'detail': 'Removal is only granted for creator'})